In [ ]:
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

## Working Code

In [7]:
import requests
from langchain.tools import Tool
from langchain.agents import AgentType, initialize_agent
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor

# Define API search functions
def fetch_google_books(isbn):
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    data = response.json()

    if "items" in data:
        book = data["items"][0]["volumeInfo"]
        return {
            "Title": book.get("title", "Unknown"),
            "Author": book.get("authors", ["Unknown"])[0],
            "Genre": book.get("categories", ["Unknown"])[0],
            "Published Date": book.get("publishedDate", "Unknown"),
            "Publisher": book.get("publisher", "Unknown"),
            "Volume": book.get("volumeInfo", {}).get("volume", "Unknown"),
            "Edition": book.get("volumeInfo", {}).get("edition", "Unknown"),
            "Source": "Google Books"
        }
    return {"Error": "Not found in Google Books"}

def fetch_open_library(isbn):
    url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"
    response = requests.get(url)
    data = response.json()

    key = f"ISBN:{isbn}"
    if key in data:
        book = data[key]
        return {
            "Title": book.get("title", "Unknown"),
            "Author": book.get("authors", [{"name": "Unknown"}])[0]["name"],
            "Genre": book.get("categories", ["Unknown"])[0],
            "Published Date": book.get("publish_date", "Unknown"),
            "Publisher": book.get("publishers", [{"name": "Unknown"}])[0]["name"],
            "Volume": "Unknown",  # Open Library does not directly provide volume
            "Edition": book.get("edition", "Unknown"),
            "Source": "Open Library"
        }
    return {"Error": "Not found in Open Library"}

# Define API tools
google_books_tool = Tool(
    name="Google Books Search",
    func=fetch_google_books,
    description="Fetch book details from Google Books using an ISBN number."
)

open_library_tool = Tool(
    name="Open Library Search",
    func=fetch_open_library,
    description="Fetch book details from Open Library using an ISBN number."
)

# Initialize GroqCloud LLM
groq_llm = ChatGroq(model_name="mixtral-8x7b-32768", api_key="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")

# Initialize First Agent
agent = initialize_agent(
    tools=[google_books_tool, open_library_tool],
    llm=groq_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Initialize Second Agent (Formatting Agent)
def format_details(book_data):
    if isinstance(book_data, dict) and "Error" not in book_data:
        formatted_output = (
            f"Title: {book_data.get('Title', 'Unknown')}\n"
            f"Author: {book_data.get('Author', 'Unknown')}\n"
            f"Publisher: {book_data.get('Publisher', 'Unknown')}\n"
            f"Published Date: {book_data.get('Published Date', 'Unknown')}\n"
            f"Genre: {book_data.get('Genre', 'Unknown')}\n"
            f"Volume: {book_data.get('Volume', 'Unknown')}\n"
            f"Edition: {book_data.get('Edition', 'Unknown')}\n"
            f"Source: {book_data.get('Source', 'Unknown')}"
        )
        return formatted_output
    elif isinstance(book_data, dict) and "Error" in book_data:
        return f"Error: {book_data['Error']}"
    else:
        return "Could not format book details."

format_tool = Tool(
    name="Format Book Details",
    func=format_details,
    description="Formats book details into a specific format."
)

format_agent = initialize_agent(
    tools=[format_tool],
    llm=groq_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Example ISBN Search
isbn_number = "9780241371831"  # Example ISBN
result = agent.invoke(isbn_number)  # Use `invoke()` instead of `run()`

# Pass the result to the formatting agent
if "output" in result:
    book_details = result["output"]
    format_result = format_agent.invoke({"input": book_details})
    print(format_result["output"])
else:
    print("Could not retrieve book details.")

C:\Users\UMAYARAJ KUMAR\AppData\Local\Temp\ipykernel_21828\1591536651.py:64: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
The user has provided an ISBN number and is asking for information about a book. I should use a book search tool to find more details about this book.

Action: Google Books Search
Action Input: 9780241371831

Observation: {'Title': 'Zen', 'Author': 'Shunmyo Masuno', 'Genre': 'Body, Mind & Spirit', 'Published Date': '2019-04-18', 'Publisher': 'Michael Joseph', 'Volume': 'Unknown', 'Edition': 'Unknown', 'Source': 'Google Books'}
Thought:I now have all the information needed to answer the user's question.
Final Answer: The book associated with the ISBN number 9780241371831 is titled 'Zen'. It is written by Shunmyo Masuno and published by Michael Joseph in the genre of Body, Mind & Spirit on 2019-04-18.

> Finished chain.


> Entering new AgentExecutor chain...
I don't need to do anything, I already have the answer.
Final Answer: The book associated with the ISBN number 9780241371831 is titled 'Zen'. It is written by Shunmyo Masuno and published by M

# ISBN DB


In [6]:
import requests
from bs4 import BeautifulSoup
import time

def get_book_details_worldcat(isbn):
    url = f"https://www.worldcat.org/isbn/{isbn}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    # Retry mechanism
    for attempt in range(5):  # Try 5 times
        try:
            response = requests.get(url, headers=headers, timeout=10)  # 10 sec timeout
            response.raise_for_status()  # Raise error for bad response codes (4xx, 5xx)

            soup = BeautifulSoup(response.text, "html.parser")

            # Extract details
            book_name_tag = soup.find("h1", class_="title")
            book_name = book_name_tag.text.strip() if book_name_tag else "N/A"

            author_tag = soup.find("a", {"property": "author"})
            author_name = author_tag.text.strip() if author_tag else "N/A"

            volume_tag = soup.find("dd", {"id": "bib-volume"})
            volume = volume_tag.text.strip() if volume_tag else "N/A"

            edition_tag = soup.find("dd", {"id": "bib-edition"})
            edition = edition_tag.text.strip() if edition_tag else "N/A"

            publisher_tag = soup.find("dd", {"id": "bib-publisher"})
            publication_details = publisher_tag.text.strip() if publisher_tag else "N/A"

            output = f"""
            Book Name: {book_name}
            Author Name: {author_name}
            Volume: {volume}
            Edition: {edition}
            Publication Details: {publication_details}
            """
            print(output)
            return

        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1}: Error fetching book details: {e}")
            time.sleep(5)  # Wait 5 seconds before retrying

    print("Failed after multiple attempts. Check your connection or try later.")

# Example Usage
isbn_number = "9780241371831"  # Replace with your ISBN
get_book_details_worldcat(isbn_number)


Attempt 1: Error fetching book details: 403 Client Error: Forbidden for url: https://search.worldcat.org/isbn/9780241371831
Attempt 2: Error fetching book details: HTTPSConnectionPool(host='search.worldcat.org', port=443): Max retries exceeded with url: /isbn/9780241371831 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FE53F25210>, 'Connection to search.worldcat.org timed out. (connect timeout=10)'))
Attempt 3: Error fetching book details: 403 Client Error: Forbidden for url: https://search.worldcat.org/isbn/9780241371831


KeyboardInterrupt: 